# [Best time to buy and sell stock](https://leetcode.com/problems/best-time-to-buy-and-sell-stock-iii/)

- Have an array of stock prices
- Return the maximum profit
    + In K=2 buy-sell turns
    + Must buy before selling
    + Must sell before buying again

#### Example 1

```
Input: prices = [3,3,5,0,0,3,1,4]
Output: 6
Explanation: Buy on day 4 (price = 0) and sell on day 6 (price = 3), profit = 3-0 = 3.
Then buy on day 7 (price = 1) and sell on day 8 (price = 4), profit = 4-1 = 3.
```

#### Example 2

```
Input: prices = [1,2,3,4,5]
Output: 4
Explanation: Buy on day 1 (price = 1) and sell on day 5 (price = 5), profit = 5-1 = 4.
Note that you cannot buy on day 1, buy on day 2 and sell them later, as you are engaging multiple transactions at the same time. You must sell before buying again.
```

#### Example 3

```
Input: prices = [7,6,4,3,1]
Output: 0
Explanation: In this case, no transaction is done, i.e. max profit = 0.
```

# Solution

#### BottomUP Initial O($N^2K$)

- Consider buying at day `j` - selling at day `i` at `k`-th transaction
    + `dp[i][k] = max(dp[j-1][k-1] + A[i]-A[j])`, $\forall j < i$

<img src="img/21.jpg" width="600"/>




```C++
class Solution {
public:
    int maxProfit(vector<int>& A) {
        int N = A.size();

        // dp[k][i] = Optimal solution at i day, k transactions
        vector<vector<int>> dp(3, vector<int>(N, 0));

        for(int i=1; i<N; ++i) {
            for(int j=0; j<i; ++j) {
                for(int k=1; k<=2; ++k) {
                    dp[k][i] = j==0 ? max(dp[k][i], A[i] - A[j]): max(dp[k][i], dp[k-1][j-1] + A[i]-A[j]);

                    // Simplify from dp[k][i] = max(dp[k][i-1], dp[k-1][i-1])
                    dp[k][i] = max(dp[k][i], dp[k][i-1]);
                }
            }
        }
        return dp[2][N-1];
    }
};
```

#### BottomUp O(NK)
- Since j < i, $\forall j$ we can update j and i at the same time
- Break `dp[i][k] = max(dp[j-1][k-1] + A[i]-A[j])` into
    - `dp_j[k][j] = max(dp_i[k-1][i-1] - A[j])`
    - `dp_i[k][i] = max(A[i] + dp_j[k][i-1])`

```C++
class Solution {
public:
    int maxProfit(vector<int>& A) {
        int N = A.size();

        vector<vector<int>> dp_j(3, vector<int>(N, -2e5));
        vector<vector<int>> dp_i(3, vector<int>(N, 0));

        for(int i=0; i<N; ++i) {
            for(int k=1; k<=2; ++k) {
                // dp for j
                dp_j[k][i] = i==0 ? -A[i]: max(dp_j[k][i], dp_i[k-1][i-1] - A[i]);
                if(i>0) dp_j[k][i] = max(dp_j[k][i], dp_j[k][i-1]);

                // dp for i
                dp_i[k][i] = i==0 ? 0: max(dp_i[k][i], A[i] + dp_j[k][i-1]);
                if(i>0) dp_i[k][i] = max(dp_i[k][i], dp_i[k][i-1]);
            }
        }
        return dp_i[2][N-1];
    }
};
```


#### BottomUp O(NK) - Optimize space complexity
- Optimize dp_j

```C++
class Solution {
public:
    int maxProfit(vector<int>& A) {
        int N = A.size();

        vector<int> dp_j(3, INT_MAX);
        vector<vector<int>> dp_i(3, vector<int>(N, 0));

        for(int i=0; i<N; ++i) {
            for(int k=1; k<=2; ++k) {
                dp_j[k] = i==0 ? A[i] :min(dp_j[k], A[i] - dp_i[k-1][i-1]);
                dp_i[k][i] = max(dp_i[k][i], A[i] - dp_j[k]);

                if(i>0) dp_i[k][i] = max(dp_i[k][i], dp_i[k][i-1]);
            }
        }
        return dp_i[2][N-1];
    }
};
```
- Optimize dp_j, dp_i

```C++
class Solution {
public:
    int maxProfit(vector<int>& A) {
        int N = A.size();

        vector<int> dp_j(3, INT_MAX);
        vector<int> dp_i(3, 0);

        for(int i=0; i<N; ++i) {
            for(int k=1; k<=2; ++k) {
                dp_j[k] = i==0 ? A[i] :min(dp_j[k], A[i] - dp_i[k-1]);
                dp_i[k] = max(dp_i[k], A[i] - dp_j[k]);

                if(i>0) dp_i[k] = max(dp_i[k], dp_i[k]);
            }
        }
        return dp_i[2];
    }
};
```


#### State machine  O(NK)

<img src="img/20.png" width="900"/>

```cpp
class Solution {
public:
    int maxProfit(vector<int>& A) {
        if(A.empty()) return 0;
        int N = A.size();

        vector<int> dp(5, INT_MIN);

        for(int i=0; i<N; ++i) {
            dp[0] = max(dp[0], 0);
            dp[1] = max(dp[1], dp[0] - A[i]);
            dp[2] = max(dp[2], dp[1] + A[i]);
            dp[3] = max(dp[3], dp[2] - A[i]);
            dp[4] = max(dp[4], dp[3] + A[i]);
        }

        return dp[4];
    }
};
```

# [Expand to K times buy-sell](https://leetcode.com/problems/best-time-to-buy-and-sell-stock-iv/)


#### State machine O(NK)
```cpp
class Solution {
public:
    int maxProfit(int K, vector<int>& A) {
        if(A.empty()) return 0;
        int N = A.size();
        
        // Special case to optimize space complexity
        if(K>N/2) {
            int ans = 0;
            for(int i=1; i<N;++i) {
                ans += max(A[i] - A[i-1], 0);
            }
            return ans;
        }

        // Normal case
        vector<vector<int>> dp(K+1, vector<int>(3 ,INT_MIN));
        dp[0][1] = 0;
        for(int i=0; i<N; ++i) {
            for(int k=1; k<=K; ++k) {
                dp[k][0] = max(dp[k][0], dp[k-1][1] - A[i]);
                dp[k][1] = max(dp[k][1], dp[k][0] + A[i]);
            }
        }
        return dp[K][1];
    }
};
```